In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns
from scipy.stats import norm
from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.preprocessing import StandardScaler
from scipy import stats

train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train_ID = train['Id']
test_ID = test['Id']
train.drop("Id", axis = 1, inplace = True)
test.drop("Id", axis = 1, inplace = True)

# df is the merged dataset (train + test)
df = pd.concat((train, test)).reset_index(drop=True)

/Users/chloeymoon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



## 2. Overall EDA (Distribution by variable)

In [12]:
# helper function to visualize distribution
def visdist(df,col):
    plt.subplot(1, 2, 1)
    plt.hist(df[col])
    plt.subplot(1, 2, 2)
    plt.scatter(df[col],df['SalePrice'])
    plt.tight_layout()
    plt.show()
    
cat_colnames = df.select_dtypes(include=['object']).columns
#print(cat_colnames, len(cat_colnames))
num_colnames = df.select_dtypes(exclude = ["object"]).columns
#print(num_colnames,len(num_colnames))

## 3. Data Cleaning: Missing Value Imputation

In [13]:
def clean(df):
    # **** what to do with Year and Month?
    for var in ['MSSubClass']:
        # later on, change to the actual string values
        df[var] = df[var].apply(str)
        
    # no garage, no bathrooms, etc., based on data description?  
    for col in ('GarageYrBlt', 'GarageArea', 'GarageCars','MasVnrArea','BsmtFinSF1','BsmtFinSF2'
           ,'BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BsmtUnfSF','TotalBsmtSF'):
        df[col] = df[col].fillna(0).astype(int)
        
    # Replacing missing data with None, based on data description
    for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','BsmtQual',
            'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',"PoolQC"
           ,'Alley','Fence','MiscFeature','FireplaceQu','MasVnrType','Utilities']:
        df[col] = df[col].fillna('None')
            
    # Home functionality (Assume typical unless deductions are warranted): 
    # data description says NA means typical
    df['Functional'] = df['Functional'].fillna('Typ')
    
    # lot frontage: correlated with lot area (0.48) and neighborhoods (domain expertise)
    df = cleanLotFrontage(df)

    # ****** MSZoning, grouped by MSSubClass but I'm not exactly sure this is valid
    # did chi square test, wasn't too helpful, so many other categorical variables associated as much
    df['MSZoning'] = df.groupby('MSSubClass')['MSZoning'].transform(lambda x: x.fillna(x.mode()[0]))
    
    common_vars = ['Exterior1st','Exterior2nd','SaleType','Electrical','KitchenQual']
    for var in common_vars:
        df[var] = df[var].fillna(df[var].mode()[0])
    
    return df

def cleanLotFrontage(df):
    df['LotArea_bin'] = pd.cut(df['LotArea'],50).apply(lambda x: x.mid)
    df['Lotfrontage_grouped'] = df.groupby(['Neighborhood','LotArea_bin'])['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    df['Lotfrontage_grouped'] = df.groupby(['Neighborhood'])['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    #df['LotFrontage_neigh'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    #df['LotFrontage_lotarea'] = df.groupby('LotArea')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
    df['LotFrontage'] = df['Lotfrontage_grouped']
    df = df.drop(columns=['LotArea_bin','Lotfrontage_grouped']) 
    return df

In [14]:
def computeMissingness(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    # df.boxplot(column='LotFrontage',by='Neighborhood')
    # plt.scatter(x=df.LotArea,y=df.LotFrontage)
    return missing_data

In [17]:
# clean the data and compute the % of missingness to confirm
cleaned = clean(df)
computeMissingness(cleaned).head(10)

,Total,Percent
SalePrice,1459,0.499829
Electrical,0,0.000000
ExterCond,0,0.000000
ExterQual,0,0.000000
Exterior1st,0,0.000000
Exterior2nd,0,0.000000
Fence,0,0.000000
FireplaceQu,0,0.000000
Fireplaces,0,0.000000
Foundation,0,0.000000


### Re-run missing data to see if there is any missing data 

In [16]:
computeMissingness(df)
# so we're all set!

,Total,Percent
PoolQC,2909,0.996574
MiscFeature,2814,0.964029
Alley,2721,0.932169
Fence,2348,0.804385
SalePrice,1459,0.499829
FireplaceQu,1420,0.486468
LotFrontage,486,0.166495
GarageQual,159,0.054471
GarageYrBlt,159,0.054471
GarageFinish,159,0.054471


### Check for Skewness for Numerical Variable

In [19]:
categorical_features = df.select_dtypes(include=['object']).columns
print(categorical_features)
numerical_features = df.select_dtypes(exclude = ["object"]).columns
print(numerical_features)

print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))
feat_num = df[numerical_features]
feat_cat = df[categorical_features]


skewness = feat_num.apply(lambda x: skew(x))
skewness.sort_values(ascending=False)

Index(['Alley', 'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'BsmtQual', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd',
       'Fence', 'FireplaceQu', 'Foundation', 'Functional', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'Heating', 'HeatingQC',
       'HouseStyle', 'KitchenQual', 'LandContour', 'LandSlope', 'LotConfig',
       'LotShape', 'MSSubClass', 'MSZoning', 'MasVnrType', 'MiscFeature',
       'Neighborhood', 'PavedDrive', 'PoolQC', 'RoofMatl', 'RoofStyle',
       'SaleCondition', 'SaleType', 'Street', 'Utilities'],
      dtype='object')
Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BedroomAbvGr', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtUnfSF',
       'EnclosedPorch', 'Fireplaces', 'FullBath', 'GarageArea', 'GarageCars',
       'GarageYrBlt', 'GrLivArea', 'HalfBath', 'KitchenAbvGr', 'LotArea',
       'LotFrontage', 'LowQua

MiscVal                21.947195
PoolArea               16.898328
LotArea                12.822431
LotArea_bin            12.479853
LowQualFinSF           12.088761
3SsnPorch              11.376065
KitchenAbvGr            4.302254
BsmtFinSF2              4.146143
EnclosedPorch           4.003891
ScreenPorch             3.946694
BsmtHalfBath            3.931594
MasVnrArea              2.613592
OpenPorchSF             2.535114
WoodDeckSF              1.842433
LotFrontage             1.505704
Lotfrontage_grouped     1.505704
1stFlrSF                1.469604
BsmtFinSF1              1.425230
GrLivArea               1.269358
TotalBsmtSF             1.156894
BsmtUnfSF               0.919339
2ndFlrSF                0.861675
TotRmsAbvGrd            0.758367
Fireplaces              0.733495
HalfBath                0.694566
BsmtFullBath            0.624832
OverallCond             0.570312
BedroomAbvGr            0.326324
GarageArea              0.239257
OverallQual             0.197110
MoSold    

In [20]:
# we need to transform those numerical features where skewness is > 0.5
# abs(skewness)>1  highly sknewed 
# 1 > abs(skewness) > 0.5 moderately sknewed
# we are taking the conservative approach and adjusting for moderately skewed 

skewness = skewness[abs(skewness) > 0.5]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))
print("Mean skewnees: {}".format(np.mean(skewness)))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    feat_num[feat] = boxcox1p(feat_num[feat], boxcox_normmax(feat_num[feat] + 1))
    df[feat] = boxcox1p(df[feat], boxcox_normmax(df[feat] + 1))
    
    
from scipy.stats import skew 
skewness.sort_values(ascending=False)

There are 29 skewed numerical features to Box Cox transform
Mean skewnees: 4.273221297187206


/Users/chloeymoon/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3038: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
/Users/chloeymoon/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


TypeError: Series cannot perform the operation +

## 4. Featuring engineering 

In [ ]:
#  Adding total sqfootage feature 
df['TotalSF']=df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']

#  Adding total bathrooms feature
df['Total_Bathrooms'] = (df['FullBath'] + (0.5 * df['HalfBath']) +
                               df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath']))

#  Adding total porch sqfootage feature
df['Total_porch_sf'] = (df['OpenPorchSF'] + df['3SsnPorch'] +
                              df['EnclosedPorch'] + df['ScreenPorch'] +
                              df['WoodDeckSF'])

In [ ]:
#  Adding total sqfootage feature 
df['TotalSF']=df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']

#  Adding total bathrooms feature
df['Total_Bathrooms'] = (df['FullBath'] + (0.5 * df['HalfBath']) +
                               df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath']))

#  Adding total porch sqfootage feature
df['Total_porch_sf'] = (df['OpenPorchSF'] + df['3SsnPorch'] +
                              df['EnclosedPorch'] + df['ScreenPorch'] +
                              df['WoodDeckSF'])

In [ ]:
#  Adding total sqfootage feature 
df['TotalSF']=df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']

#  Adding total bathrooms feature
df['Total_Bathrooms'] = (df['FullBath'] + (0.5 * df['HalfBath']) +
                               df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath']))

#  Adding total porch sqfootage feature
df['Total_porch_sf'] = (df['OpenPorchSF'] + df['3SsnPorch'] +
                              df['EnclosedPorch'] + df['ScreenPorch'] +
                              df['WoodDeckSF'])